In [32]:
from tkinter import *
import requests
import json
import datetime

class WeatherApp:
    def __init__(self, root):
        self.root = root
        self.setup_ui()

    def setup_ui(self):
        self.root.title("Weather App")
        self.root.geometry("600x500")

        # City label and input box
        self.city_label = Label(self.root, text="Write your city", font=("Times New Roman", 12))
        self.city_label.grid(row=0, column=0)

        self.city_entry = Entry(self.root, width=40)
        self.city_entry.grid(row=0, column=1)

        # Country label and input box
        self.country_label = Label(self.root, text="Write your country code", font=("Times New Roman", 12))
        self.country_label.grid(row=1, column=0)

        self.country_entry = Entry(self.root, width=40)
        self.country_entry.grid(row=1, column=1)

        # Button to start the base weather app
        self.weather_button = Button(self.root, text="Basic weather", font=("Times New Roman", 12),
                                     command=self.get_weather)
        self.weather_button.grid(row=2, column=0, columnspan=2, pady=10, padx=10, ipadx=200)

        # Button to start the detailed weather app
        self.detailed_button = Button(self.root, text="Detailed Weather", font=("Times New Roman", 12),
                                      command=self.show_detailed_weather)
        self.detailed_button.grid(row=3, column=0, pady=10, padx=10, ipadx=100)

        self.output_label = Label(self.root, text="", font=("Times New Roman", 12))
        self.output_label.grid(row=4, column=0, columnspan=2)

        self.root.mainloop()

    def get_weather(self):
        # API key
        api_key = "0e278eb7bf7a1a9bc0d633da8c318d6d"

        # API request
        city = self.city_entry.get()
        country = self.country_entry.get()
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city},{country}&appid={api_key}"
        response = requests.get(url)

        # Parse the JSON data
        self.data = json.loads(response.text)

        # Extract relevant weather information
        description = str(self.data["weather"][0]["description"])
        temp = str(round(self.data["main"]["temp"] - 273.15, 2))
        feels_like = str(round(self.data["main"]["feels_like"] - 273.15, 2))
        temp_min = str(round(self.data["main"]["temp_min"] - 273.15, 2))
        temp_max = str(round(self.data["main"]["temp_max"] - 273.15, 2))
        pressure = str(self.data["main"]["pressure"])
        humidity = str(self.data["main"]["humidity"])
        visibility = str(self.data["visibility"])
        clouds = str(self.data["clouds"]["all"])
        wind = str(self.data["wind"]["speed"])
        sunrise = str(datetime.datetime.fromtimestamp(self.data["sys"]["sunrise"]).strftime("%H:%M:%S"))
        sunset = str(datetime.datetime.fromtimestamp(self.data["sys"]["sunset"]).strftime("%H:%M:%S"))
        dataCity = str(self.data["name"])
        dataCountry = str(self.data["sys"]["country"])
        dt = str(datetime.datetime.fromtimestamp(self.data["dt"]).strftime("%H:%M:%S %d.%m.%Y"))

        # Printing out the output
        outputText = "description: " + description + "\n" +"temperature: " + temp + " °C" + "\n" + \
        "feels like: " + feels_like + " °C" + "\n" + "minimum temperature: " + temp_min + " °C" + "\n" + \
        "maximum temperature: " + temp_max + " °C" + "\n" + "pressure: " + pressure+ " hPa" + "\n" + \
        "humidity: " + humidity+ " %" + "\n" + "visibility: " + visibility+ " m (maximum is 10000 m)" + "\n" +\
        "cloudiness: " + clouds+ " %" + "\n" + "wind speed: " + wind + "\n" + "sunrise: " + sunrise +"\n" +\
        "sunset: " + sunset +"\n" +"city: " + dataCity +"\n" +"country: " + dataCountry +"\n" +\
        "data downloaded at this time: " + dt
        
        self.output_label.config(text=outputText)

    def show_detailed_weather(self):
        city = self.city_entry.get()
        country = self.country_entry.get()
        data = self.data

        # Create a new window for the detailed weather forecast
        detailed_window = Toplevel(self.root)
        detailed_app = DetailedWeather(detailed_window, city, country,data)

class DetailedWeather:
    def __init__(self, root, city, country, data):
        self.root = root
        self.city = city
        self.country = country
        self.data = data
        self.output_detailed_label = Label(self.root, text="", font=("Times New Roman", 12))
        self.setup_ui()
        

    def setup_ui(self):
        self.root.title("Detailed Weather")
        self.root.geometry("400x300")

        # Display the city and country
        location_label = Label(self.root, text=f"Weather forecast for {self.city}, {self.country}",
                               font=("Times New Roman", 12))
        location_label.grid(row=0, column=0)

        # Retrieve and display the detailed forecast
        detailed_label = Label(self.root, text=self.get_detailed_weather(), font=("Times New Roman", 12))
        detailed_label.grid(row=1, column=0)


    def get_detailed_weather(self):

        if "sea_level" in self.data["main"]:
            # data not always available
            seaPressure = str(self.data["main"]["sea_level"])
            groundPressure = str(self.data["main"]["grnd_level"])
            pressure_text = "Atmospheric pressure on the sea level: " + seaPressure + " hPa\n" + \
                            "Atmospheric pressure on the ground level: " + groundPressure + " hPa"
        else:
            pressure_text = "Pressure data for sea level and ground level is unavailable"

        if "rain" in self.data:
            rain1h = str(self.data["rain"]["1h"])
            rain3h = str(self.data["rain"]["3h"])
            rain_text = "Rain volume for the last 1 hour: " + rain1h + " mm\n" + \
                        "Rain volume for the last 3 hours: " + rain3h + " mm"
        else:
            rain_text = "Rain data unavailable"

        if "snow" in self.data:
            snow1h = str(self.data["snow"]["1h"])
            snow3h = str(self.data["snow"]["3h"])
            snow_text = "Snow volume for the last 1 hour: " + snow1h + " mm" + "\n" + \
                "snow volume for the last 3 hour: " + snow3h +" mm"
        else:
            snow_text = "Snow data unavailable"
        
        detailedText = pressure_text +"\n"+ rain_text +"\n"+ snow_text

        self.output_detailed_label.config(text=detailedText)
        self.output_detailed_label.grid(row=1, column=0)

root = Tk()
weather_app = WeatherApp(root)  # Base weather app
root.mainloop()


{'coord': {'lon': 13.7334, 'lat': 50.1037}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 290.4, 'feels_like': 289.38, 'temp_min': 289.18, 'temp_max': 293, 'pressure': 1022, 'humidity': 46, 'sea_level': 1022, 'grnd_level': 984}, 'visibility': 10000, 'wind': {'speed': 4.09, 'deg': 29, 'gust': 4.32}, 'clouds': {'all': 49}, 'dt': 1685433932, 'sys': {'type': 2, 'id': 2039282, 'country': 'CZ', 'sunrise': 1685415751, 'sunset': 1685473375}, 'timezone': 7200, 'id': 3067051, 'name': 'Rakovník', 'cod': 200}
